![](../storage/banners/28_observability.png)

##### **<u>Свои догадки</u>**

###### Попытка вывода симметрии у решения - Частные случаи

In [11]:
from sympy import *
import sympy
from cosmetic import *

def add_extraname(s, extraname):
    return " ".join([i+extraname for i in s.split()])

def get_x_y_z(case: dict, extraname: str = "", planar: bool = False, test_xz: bool = False):
    x0, y0, z0, vx0, vy0, vz0 = var(add_extraname(s="x_0 y_0 z_0 v^x_0 v^y_0 v^z_0", extraname=extraname))
    X0, Y0, Z0, VX0, VY0, VZ0 = var(add_extraname(s="X_0 Y_0 Z_0 V^X_0 V^Y_0 V^Z_0", extraname=extraname))
    c, rho, m, M, s, S, v_orb = var(add_extraname(s="C rho m M s S v_{orb}", extraname=extraname))
    t, w0 = var("t w_0")

    r, R = 3*[None], 3*[None]
    
    if case['C1 = 0']:
        vx0 = - 2 * z0 * w0
        VX0 = - 2 * Z0 * w0
    if not case['CubeSat motion']:
        X0, Y0, Z0, VX0, VY0, VZ0 = [0]*6
    if planar:
        y0, Y0, vy0, VY0 = [0]*4

    for _x0, _y0, _z0, _vx0, _vy0, _vz0, _r, _m, _s in zip((x0, X0), (y0, Y0), (z0, Z0), (vx0, VX0), (vy0, VY0), (vz0, VZ0), 
                                                           (r, R), (m, M), (s, S)):
        C_1 = 2 * _z0 + _vx0 / w0
        C_2 = _vz0 / w0
        C_3 = -3 * _z0 - 2 * _vx0 / w0
        C_4 = _x0 - 2 * _vz0 / w0
        C_5 = _vy0 / w0
        C_6 = _y0
        
        tmp = (-t**2 / 2 * c * rho / _m * _s * v_orb**2) if case['Aero'] else 0
        
        _r[0] = C_4 - 3*C_1*w0*t + 2*C_2*cos(w0*t) -2*C_3*sin(w0*t) + tmp
        _r[1] = C_5*sin(w0*t) + C_6*cos(w0*t)
        _r[2] = 2*C_1 + C_2*sin(w0*t) + C_3*cos(w0*t)
    
    return {'w0': w0, 't': t,
            'x0': x0, 'y0': y0, 'z0': z0, 'vx0': vx0, 'vy0': vy0, 'vz0': vz0,
            'x': r[0], 'y': r[1], 'z': r[2], 
            'X': R[0], 'Y': R[1], 'Z': R[2]}

def get_measurements(case: dict, params: dict):
    if case['antenna type'] == 'isotropic':
        measurements = [(params['X'] - params['x'])**2 + 
                        (params['Y'] - params['y'])**2 + 
                        (params['Z'] - params['z'])**2]

    return measurements

def get_discrepancy_invert_param(measurements, params: dict, variables: list) -> list:
    global counter
    subses = []
    for v in variables:
        subses.append((params[v], -params[v]))

    anw = []
    for i, m in zip(range(len(measurements)), measurements):
        tmp = (m - m.subs(subses)).simplify()
        anw.append(tmp)
        if isinstance(tmp, sympy.core.numbers.Zero):
            print(f"По параметрам\033[1m", *variables, f"\033[0mу измерения №{i+1} есть зеркальная симметрия")
            counter *= 2
    return anw

def print_symmetry_problem_result():
    global counter
    if counter == 1:
        my_print(f"Решение единственное: {counter}!", bold=True, color="g")
    else:
        my_print(f"Симметричных решений: {counter}", bold=True, color="b")


def calculate_symmetry(motion_case, measurement_case, v_list):
    global counter
    counter = 1
    params = get_x_y_z(case=motion_case)
    measurements = get_measurements(case=measurement_case, params=params)
    for variables in v_list:
        _ = get_discrepancy_invert_param(measurements=measurements, variables=variables, params=params)
    print_symmetry_problem_result()

###### 0. Поиск решения

>> Уравнения:
$$\ddot{x} = -2 \omega_0 \upsilon_z,$$
$$\ddot{y} = -\omega_0^2 y,$$
$$\ddot{z} = 2 \omega_0 \upsilon_x + 3 \omega_0^2 z.$$

> Движение по $y$ отделимо <br>
> Возможна симметрия по одновременному изменению положений и скоростей по $x, z$ <br>

In [13]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

params = get_x_y_z(case=motion_case)
measurements = get_measurements(case=measurement_case, params=params)

# params_wrong = get_x_y_z(case=motion_case, planar=False, extraname="^w")
# measurements_wrong = get_measurements(case=measurement_case, params=params_wrong)
measurements[0]

(-v^y_0*sin(t*w_0)/w_0 - y_0*cos(t*w_0))**2 + (-2*v^x_0/w_0 - v^z_0*sin(t*w_0)/w_0 - 4*z_0 - (-2*v^x_0/w_0 - 3*z_0)*cos(t*w_0))**2 + (t*w_0*(3*v^x_0/w_0 + 6*z_0) - 2*v^z_0*cos(t*w_0)/w_0 + 2*v^z_0/w_0 - x_0 + (-4*v^x_0/w_0 - 6*z_0)*sin(t*w_0))**2

In [16]:
grad_vec = {}
for v in ["x0", "y0", "z0", "vx0", "vy0", "vz0"]:
    grad_vec[v] = measurements[0].diff(params[v])

###### 1. ХКУ, 1к-1ч, изотропные антенны, без определение углового движения
> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br>
> Итого: 4 симметричных решения

In [117]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vx0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 4


In [118]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

Решение единственное: 1!


In [119]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 8


In [120]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


###### 2. ХКУ + аэро, 1к-1ч, изотропные антенны, без определение углового движения
> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br> 
> Итого: 4 симметричных решения

In [121]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


In [122]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)


KeyboardInterrupt



In [105]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
По параметрам y0 vy0 у измерения №1 есть симметрия
Симметричных решений: 4


In [106]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
Симметричных решений: 2


###### <span style="color:#2c3e50">Минимальное кол-во материнских аппаратов для наблюдаемости</span> 

In [9]:
def get_min_chief_amount(fn: int, motion: str, antenna_config: dict) -> int:
    """
    :param fn: кол-во дочерних КА (НЕ УЧИТЫВАЕТСЯ)
    :param motion: По каким координатам ОСК движение КА
    :param antenna_config: типы антенн материнских и дочерних КА
    :return: Кол-во материнских КА для наблюдаемости на НОО (околокруговые)
    """
    for cn in range(1, 10):
        n_symmetry_solution = [1, 1, 1]
        ############################
        # Увеличение симметричных решений
        if antenna_config['c-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        if antenna_config['d-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        
        # Уменьшение симметричных решений
        if not antenna_config['c-multy-send'] and not antenna_config['d-multy-take']:
            if cn == 4:
                for i in range(3):
                    n_symmetry_solution[i] /= 2
        ############################
        if sum(n_symmetry_solution) == 3:
            return cn
    print("Не нашлось такого количества! Измени параметры")
    return -1
    
cn = get_min_chief_amount(fn=1, motion="xyz", 
                          antenna_config={'c-type': ["изотропные"][0], 
                                          'd-type': ["изотропные"][0],
                                          'c-multy-send': False,
                                          'c-multy-take': False,
                                          'd-multy-send': False,
                                          'd-multy-take': False,})
print(f"Минимальное кол-во материнских аппаратов для наблюдаемости: \033[1m{cn}\033[0m")

Не нашлось такого количества! Измени параметры
Минимальное кол-во материнских аппаратов для наблюдаемости: -1


###### Закрутка чипсата

##### **<u>Согласно статье</u> 038 (Shauying R.K.) Observability of Nonlinear Systems**

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">055 (Yujiro Inowe) On the Observability of Autonomous Nonlinear Systems</span> <span style="color:#0ab49a">надо просто проверить ранг во всём $R^n$!</span>    $rg\frac{\partial \boldsymbol{H}_d}{\partial \boldsymbol{x}} (\boldsymbol{x}) = n \hskip20px \forall x \in R^n,$
$$d = n \frac{n +3}{2}.$$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">051 (Andrew J. Whalen) Observability and Controllability of Nonlinear Networks The Role of Symmetry</span> <span style="color:#0ab49a">надо проверить</span> $\delta(x) = \frac{|\sigma_{min}[O^T O]|}{|\sigma_{max}[O^T O]|}.$

###### <u>Инициализация</u>

In [6]:
from common_func import *

o, num_params, t, ω, μ, ρ, r_orb, v_orb = init_symbol_params()

Параметры не могут быть загружены! Нет файла: kiamformation/data/config_choose.csv
Матрицы Ф:(6, 6), Q:(3, 3), P:(6, 6), D:(6, 3)
Высота орбиты: 400 км
Период орбиты: 1.54 часов
Плотность атмосферы: 6.404751331738951e-12 кг/м³


###### <u>Алгоритм</u>

In [7]:
def my_diff(expr, power: int = 1, n_c=1, n_d=1):
    """
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной
    """    
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for j in range(3):
        for i in range(n_d):
            subses += [(Derivative(r_d[i][j], t), dr_d[i][j])]
            subses += [(Derivative(v_d[i][j], t), dv_d[i][j])]
            subses += [(Derivative(q_d[i][j+1], t), dq_d[i][j+1])]
            subses += [(Derivative(ω_d[i][j], t), dω_d[i][j])]
        for i in range(n_c):
            subses += [(Derivative(r_c[i][j], t), dr_c[i][j])]
            subses += [(Derivative(v_c[i][j], t), dv_c[i][j])]
            subses += [(Derivative(q_c[i][j+1], t), dq_c[i][j+1])]
            subses += [(Derivative(ω_c[i][j], t), dω_c[i][j])]
    anw = anw.diff(t).subs(subses)
    anw.simplify()
    return anw

def SubRandParams(J, n_d: int, n_c: int, n_x: int, n_y: int, testprint: bool = False):
    """Берёт матрицу J размером n_x на n_y, подставляет случайные значения"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    # Генерация случайных параметров движения
    s_r = lambda: np.random.uniform(-100, 100)
    s_v = lambda: np.random.uniform(-10, 10)
    s_w = lambda: np.random.uniform(-1e-4, 1e-4)
    s_q = lambda: np.random.uniform(-0.5, 0.5)
    rand_params = [(ω, num_params['ω0']), (pi, np.pi), (ρ, num_params['ρ']), (v_orb, num_params['v0']), 
                   (o.f.c_resist, num_params['Cd']), (o.c.c_resist, num_params['Cc']), 
                   (o.f.mass, num_params['md']), (o.c.mass, num_params['mc']),
                   (o.f.size[0], num_params['sd'][0]), (o.f.size[1], num_params['sd'][1]),
                   (o.c.size[0], num_params['sc'][0]), (o.c.size[1], num_params['sc'][1]),
                   (o.f.J[0, 0], num_params['Jd'][0, 0]), (o.f.J[1, 1], num_params['Jd'][1, 1]), (o.f.J[2, 2], num_params['Jd'][2, 2]),
                  ] 
    for j in range(3):
        for i in range(n_d):
            rand_params.extend([(r_d[i][j], s_r()), (v_d[i][j], s_v()), (ω_d[i][j], s_w()), (q_d[i][j+1], s_q())])
        for i in range(n_c):
            rand_params.extend([(r_c[i][j], s_r()), (v_c[i][j], s_v()), (ω_c[i][j], s_w()), (q_c[i][j+1], s_q())])

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)] 
    J_numb = np.array([[0. for _ in range(n_x)] for _ in range(n_y)])
    for i in range(n_y):
        if testprint:
            print(f"    J_numb: расчёт строки : {i+1} / {n_y}")
        for j in range(n_x):
            J_numb[i][j] = float(J[i, j].subs(rand_params))
    return J_numb

def ShauyingObservabilitySufficientCondition(n_d: int, n_c: int, X: list, Y: list, testprint: bool = False, hand_written_deriv: int = None):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n_d: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени
    :param testprint: Флаг вывода экстра-информации"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    def print_and_record(report: str, lcl_txt: str):
        print(lcl_txt)
        return report + lcl_txt + "\n"
    report = kf.my_print(f"Количество кубсатов: {n_c}\nКоличество чипсатов: {n_d}\n", if_return=True, bold=True)

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    d = len(X) * (len(X) + 3) / 2  / len(Y)
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    report = print_and_record(report, txt + f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n_d)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)\nКритерий (055): происзводные порядка {d}")
    
    # Матрица наблюдаемости системы
    H = Matrix([[Y[0] for ll in range(l)] for kk in range(k)])
    H_one_line = []
    for kk in range(k):
        for ll in range(l):
            tmp = Y[ll] if kk == 0 else my_diff(H[kk - 1, ll], n_c=n_c, n_d=n_d)
            if testprint:
                print(f"_расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H[kk, ll] = tmp
            H_one_line += [tmp]
    H = Matrix(H_one_line)
    report = print_and_record(report, f"Размерность матрицы H: {shape(H)}")

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[H[i].diff(X[j]) for j in range(len(X))] for i in range(k * l)])
    report = print_and_record(report, f"Размерность матрицы J: {shape(J)}")

    # Подстановка конкретных значений
    J_numb = SubRandParams(J=J, n_c=n_c, n_d=n_d, n_x=len(X), n_y=k*l, testprint=testprint)
    _, v, _ = np.linalg.svd(J_numb.T @ J_numb)
    report = print_and_record(report, f"v = {v}")
    report = print_and_record(report, f"σₘₙ/σₘₐₓ = {np.min(v)}/{np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁵: {np.sum(v>1e-5)}/{len(v)} (статья 051)")

    # Достаточное условие
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]]} (статья 055)\n"
    txt += f"Детерминант матрицы: {np.linalg.det(J_numb)}\n" if J_numb.shape[0] == J_numb.shape[1] else ""
    if hand_written_deriv is None:
        report = print_and_record(report, txt + f"Следующие параметры не должны быть нулевыми:\n")
        d, Δ, flag, i_min = ([], [], True, -1)
        for i in range(len(X)):
            tmp = kf.matrix_minor(J_numb, i, i)
            d += [tmp if i == 0 else tmp / Δ[-1]]
            Δ += [tmp]
            report = print_and_record(report, f"Δ_{i} = {Δ[-1]}" if i == 0 else f"Δ_{i} / Δ_{i-1} = {Δ[-1]}")
        
            # Чек наблюдаемости
            if flag:
                if abs(d[-1]) < 1e-6:
                    i_min = i
                    flag = False
            if not flag:
                break
    
        # Вывод
        if flag:
            txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
        else:
            in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
            txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
        report = print_and_record(report, txt)
        return H, J, J_numb, Δ, report
    report = print_and_record(report, txt)
    return H, J, J_numb, report

def observe_system(n_c, n_d, is_c_move, is_d_qw, is_angles, is_drag, is_only_xz, testprint, hand_written_deriv):
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c
    r_d, v_d, q_d, ω_d = get_state_vector(func=kf.get_func, obj='d', n=n_d)
    r_c, v_c, q_c, ω_c = get_state_vector(func=kf.get_func, obj='c', n=n_c)
    # U_d, S_d, A_d, r0_d = kf.get_matrices(v=o.v, t=t, q=q_d)
    # U_c, S_c, A_c, r0_c = kf.get_matrices(v=o.v, t=t, q=q_c)

    o.c.n = n_c
    o.f.n = n_d

    if is_c_move:
        o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_irf = r_c, v_c, q_c, ω_c
    else:
        o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_irf = ([zeros(3, 1) for _ in range(n_c)], 
                                                  [zeros(3, 1) for _ in range(n_c)], 
                                                  [Matrix([1, 0, 0, 0]) for _ in range(n_c)], 
                                                  [Matrix([0, 0, ω]) for _ in range(n_c)])

    if is_d_qw:
        x = kf.sympy_append(kf.sympy_append(*r_d), kf.sympy_append(*[Matrix(q[1:4]) for q in q_d]), kf.sympy_append(*v_d), kf.sympy_append(*ω_d))
    else:
        x = kf.sympy_append(kf.sympy_append(*r_d), kf.sympy_append(*v_d))
    kf.my_print(f"Вектор состояния:", bold=True)
    display(x.T)
    
    o.v.GAIN_MODEL_C_N, o.v.GAIN_MODEL_F_N = (0, 0)
    o.v.init_choice_params()
    o.v.NAVIGATION_ANGLES = is_angles
    o.v.DYNAMIC_MODEL['aero drag'] = is_drag

    if is_angles:
        o.v.GAIN_MODEL_C_N = 2
        o.v.GAIN_MODEL_F_N = 2
        o.v.init_choice_params()
        o.c.gain_mode = o.v.GAIN_MODEL_C
        o.f.gain_mode = o.v.GAIN_MODEL_F
        o.v.MULTI_ANTENNA_TAKE = True
        o.v.MULTI_ANTENNA_SEND = True
    
    y, notes = kf.measure_antennas_power(c=o.c, f=o.f, v=o.v, p=o.p, j=len(x), estimated_params=x, t=t)
    kf.my_print(f"Вектор измерений:", bold=True)
    display(y)
    
    y = Matrix([y[::2]])
    kf.my_print(f"Вектор измерений (изменённый):", bold=True)
    display(y)

    if is_only_xz:
        tmp = []
        for i in range(n_d):
            tmp.append([r_d[i][0], r_d[i][2], v_d[i][0], v_d[i][2]])
        x = Matrix(tmp)
        kf.my_print(f"Вектор состояния (изменённый):", bold=True)
        display(x.T)
    
    dr_d, dv_d, dq_d, dω_d = ([0 for _ in range(n_d)] for _ in range(4))
    dr_c, dv_c, dq_c, dω_c = ([0 for _ in range(n_c)] for _ in range(4))
    for i in range(n_d):
        dr_d[i], dv_d[i] = kf.translate_rhs(vrs=o.v, obj=o.f, i=0, rv=(r_d[i], v_d[i]), w=ω, mu=μ, rho=ρ)
        dq_d[i], dω_d[i] = kf.attitude_rhs(v=o.v, obj=o.f, t=t, i=0, qw=(q_d[i], ω_d[i]))
    for i in range(n_c):
        dr_c[i], dv_c[i] = kf.translate_rhs(vrs=o.v, obj=o.c, i=0, rv=(r_c[i], v_c[i]), w=ω, mu=μ, rho=ρ)
        dq_c[i], dω_c[i] = kf.attitude_rhs(v=o.v, obj=o.c, t=t, i=0, qw=(q_c[i], ω_c[i]))
    
    return ShauyingObservabilitySufficientCondition(testprint=True, n_c=n_c, n_d=n_d, X=x, Y=y, hand_written_deriv=hand_written_deriv)

###### <u>Наблюдаемость системы</u> без аэродинамики и углового движения, антенны изотропные

In [3]:
H_1_1, J_1_1, J_numb_1_1, Δ_1_1, report_1_1 = observe_system(n_c=1, 
                                                             n_d=1, 
                                                             is_c_move=False, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=False, 
                                                             is_only_xz=False, 
                                                             testprint=True,
                                                             hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)],
[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)]])

Вектор измерений (изменённый):


Matrix([[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 1
∃ производные порядка k = 6.0 (Должна быть целой!)
Критерий (055): происзводные порядка 27.0
_расчёт матрицы H_: k=1/6, l=1/1
_расчёт матрицы H_: k=2/6, l=1/1
_расчёт матрицы H_: k=3/6, l=1/1
_расчёт матрицы H_: k=4/6, l=1/1
_расчёт матрицы H_: k=5/6, l=1/1
_расчёт матрицы H_: k=6/6, l=1/1
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
    J_numb: расчёт строки : 1 / 6
    J_numb: расчёт строки : 2 / 6
    J_numb: расчёт строки : 3 / 6
    J_numb: расчёт строки : 4 / 6
    J_numb: расчёт строки : 5 / 6
    J_numb: расчёт строки : 6 / 6
v = [1.00478867e+00 1.00002212e+00 1.81522976e-02 1.98856791e-11
 2.37559019e-17 5.40095800e-18]
σₘₙ/σₘₐₓ = 5.4009579984034685e-18/1.0047886737969498 = 5.3752178336107594e-18 | σ>10⁻⁵: 3/6 (статья 051)

Ранг матрицы: [3, 3, 4, 6, 6, 6] (статья 055)
Детерминант матрицы: 7.741822811904543e-25
Следующие параметры не должны быть 

In [8]:
H_2_1, J_2_1, J_numb_2_1, Δ_2_1, report_2_1 = observe_system(n_c=2, 
                                                             n_d=1, 
                                                             is_c_move=True, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=False, 
                                                             is_only_xz=False, 
                                                             testprint=False,
                                                             hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)]])

Вектор измерений (изменённый):


Matrix([[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2), sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)]])

Количество кубсатов: 2
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 2
∃ производные порядка k = 3.0 (Должна быть целой!)
Критерий (055): происзводные порядка 13.5
_расчёт матрицы H_: k=1/3, l=1/2
_расчёт матрицы H_: k=1/3, l=2/2
_расчёт матрицы H_: k=2/3, l=1/2
_расчёт матрицы H_: k=2/3, l=2/2
_расчёт матрицы H_: k=3/3, l=1/2
_расчёт матрицы H_: k=3/3, l=2/2
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
    J_numb: расчёт строки : 1 / 6
    J_numb: расчёт строки : 2 / 6
    J_numb: расчёт строки : 3 / 6
    J_numb: расчёт строки : 4 / 6
    J_numb: расчёт строки : 5 / 6
    J_numb: расчёт строки : 6 / 6
v = [1.92988077e+00 1.90958695e+00 1.00880593e-01 7.78097834e-02
 1.19659073e-03 2.20645298e-06]
σₘₙ/σₘₐₓ = 2.206452981417231e-06/1.9298807674138023 = 1.1433105188016656e-06 | σ>10⁻⁵: 5/6 (статья 051)

Ранг матрицы: [6, 6, 6, 6, 6, 6] (статья 055)
Детерминант матрицы: 8.739287558536382e-06
Следующие параметры не должны быть н

In [10]:
H_3_1, J_3_1, J_numb_3_1, Δ_3_1, report_3_1 = observe_system(n_c=3, 
                                                             n_d=1, 
                                                             is_c_move=True, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=False, 
                                                             is_only_xz=False, 
                                                             testprint=False,
                                                             hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2)]])

Вектор измерений (изменённый):


Matrix([[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2), sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2), sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2)]])

Количество кубсатов: 3
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 3
∃ производные порядка k = 2.0 (Должна быть целой!)
Критерий (055): происзводные порядка 9.0
_расчёт матрицы H_: k=1/2, l=1/3
_расчёт матрицы H_: k=1/2, l=2/3
_расчёт матрицы H_: k=1/2, l=3/3
_расчёт матрицы H_: k=2/2, l=1/3
_расчёт матрицы H_: k=2/2, l=2/3
_расчёт матрицы H_: k=2/2, l=3/3
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
    J_numb: расчёт строки : 1 / 6
    J_numb: расчёт строки : 2 / 6
    J_numb: расчёт строки : 3 / 6
    J_numb: расчёт строки : 4 / 6
    J_numb: расчёт строки : 5 / 6
    J_numb: расчёт строки : 6 / 6
v = [1.92608954 1.81829476 1.19125186 1.05319309 0.04695186 0.00803984]
σₘₙ/σₘₐₓ = 0.008039844240931123/1.9260895375160711 = 0.004174179904065877 | σ>10⁻⁵: 6/6 (статья 051)

Ранг матрицы: [6, 6, 6, 6, 6, 6] (статья 055)
Детерминант матрицы: 0.04072643480619379
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0213046368014

In [16]:
H_3_1, J_3_1, J_numb_3_1, Δ_3_1, report_3_1 = observe_system(n_c=6, 
                                                             n_d=1, 
                                                             is_c_move=True, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=False, 
                                                             is_only_xz=False, 
                                                             testprint=False,
                                                             hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_3^c_x(t))**2 + (r_0^d_y(t) - r_3^c_y(t))**2 + (r_0^d_z(t) - r_3^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_3^c_x(t))**2 + (r_0^d_y(t) - r_3^c_y(t))**2 + (r_0^d_z(t) - r_3^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_4^c_x(t))**2 + (r_0^d_y(t) - r_4^c_y(t))**2 + (r_0^d_z(t) - r_4^c_z(t))**2)],
[   sqrt((r_0^d_x(t) - r_4^c_x(t))**2 + (r_0^d_y(t) - r_4^c_y(t)

Вектор измерений (изменённый):


Matrix([[sqrt((-r_0^c_x(t) + r_0^d_x(t))**2 + (-r_0^c_y(t) + r_0^d_y(t))**2 + (-r_0^c_z(t) + r_0^d_z(t))**2), sqrt((r_0^d_x(t) - r_1^c_x(t))**2 + (r_0^d_y(t) - r_1^c_y(t))**2 + (r_0^d_z(t) - r_1^c_z(t))**2), sqrt((r_0^d_x(t) - r_2^c_x(t))**2 + (r_0^d_y(t) - r_2^c_y(t))**2 + (r_0^d_z(t) - r_2^c_z(t))**2), sqrt((r_0^d_x(t) - r_3^c_x(t))**2 + (r_0^d_y(t) - r_3^c_y(t))**2 + (r_0^d_z(t) - r_3^c_z(t))**2), sqrt((r_0^d_x(t) - r_4^c_x(t))**2 + (r_0^d_y(t) - r_4^c_y(t))**2 + (r_0^d_z(t) - r_4^c_z(t))**2), sqrt((r_0^d_x(t) - r_5^c_x(t))**2 + (r_0^d_y(t) - r_5^c_y(t))**2 + (r_0^d_z(t) - r_5^c_z(t))**2)]])

Количество кубсатов: 6
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 6
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 4.5
_расчёт матрицы H_: k=1/1, l=1/6
_расчёт матрицы H_: k=1/1, l=2/6
_расчёт матрицы H_: k=1/1, l=3/6
_расчёт матрицы H_: k=1/1, l=4/6
_расчёт матрицы H_: k=1/1, l=5/6
_расчёт матрицы H_: k=1/1, l=6/6
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
    J_numb: расчёт строки : 1 / 6
    J_numb: расчёт строки : 2 / 6
    J_numb: расчёт строки : 3 / 6
    J_numb: расчёт строки : 4 / 6
    J_numb: расчёт строки : 5 / 6
    J_numb: расчёт строки : 6 / 6
v = [2.69510789 2.09062384 1.21426827 0.         0.         0.        ]
σₘₙ/σₘₐₓ = 0.0/2.695107894585057 = 0.0 | σ>10⁻⁵: 3/6 (статья 051)

Ранг матрицы: [3, 3, 3, 3, 3, 3] (статья 055)
Детерминант матрицы: 0.0
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0

In [ ]:
# save_reports([report_11_1, report_11_5, report_11_11, report_11_1_, report_11_3_, report_11_7_], "AeroOn_AttitudeOff_AntennaOff")
# print(read_reports("AeroOn_AttitudeOff_AntennaOff"))

###### <u>Наблюдаемость системы</u> c аэродинамикой, без углового движения, антенны изотропные

In [ ]:
H_tmp, J_tmp, J_numb_tmp, Δ_tmp, report_tmp = observe_system(n_c=2, 
                                                             n_d=1, 
                                                             is_c_move=True, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=True, 
                                                             is_only_xz=False, 
                                                             testprint=False,
                                                             hand_written_deriv=None)

In [14]:
H_tmp, J_tmp, J_numb_tmp, Δ_tmp, report_tmp = observe_system(n_c=1, 
                                                             n_d=1, 
                                                             is_c_move=False, 
                                                             is_d_qw=False, 
                                                             is_angles=False, 
                                                             is_drag=True, 
                                                             is_only_xz=True, 
                                                             testprint=False,
                                                             hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)],
[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)]])

Вектор измерений (изменённый):


Matrix([[sqrt(r_0^d_x(t)**2 + r_0^d_y(t)**2 + r_0^d_z(t)**2)]])

Вектор состояния (изменённый):


Matrix([
[r_0^d_x(t)],
[r_0^d_z(t)],
[v_0^d_x(t)],
[v_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 1
∃ производные порядка k = 4.0 (Должна быть целой!)
Критерий (055): происзводные порядка 14.0
_расчёт матрицы H_: k=1/4, l=1/1
_расчёт матрицы H_: k=2/4, l=1/1
_расчёт матрицы H_: k=3/4, l=1/1
_расчёт матрицы H_: k=4/4, l=1/1
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)
    J_numb: расчёт строки : 1 / 4
    J_numb: расчёт строки : 2 / 4
    J_numb: расчёт строки : 3 / 4
    J_numb: расчёт строки : 4 / 4
v = [1.01005853e+00 9.91224792e-01 4.11394173e-02 3.18725872e-13]
σₘₙ/σₘₐₓ = 3.1872587164221276e-13/1.0100585309839982 = 3.155518832475087e-13 | σ>10⁻⁵: 3/4 (статья 051)

Ранг матрицы: [3, 3, 4, 4, 4, 4] (статья 055)
Детерминант матрицы: -1.1457174342670641e-07
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.00022814262825270046
Δ_1 / Δ_0 = 0.00520245822266811
Δ_2 / Δ_1 = -0.0009497291018764873
Δ_3 / Δ_2 = -0.01635263353004185

Выполнено достаточное у

###### <u>Наблюдаемость системы</u> с аэродинамикой, с угловым движением, антенны - полуволновой диполь

In [ ]:
H_3, Js_3, Jn_3, Δ_3, report_3 = observe_system(n_c=2, 
                                                n_d=1, 
                                                is_c_move=True, 
                                                is_d_qw=True, 
                                                is_angles=True, 
                                                is_drag=True, 
                                                is_only_xz=True, 
                                                testprint=False,
                                                hand_written_deriv=None)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), ω_0^d_x(t), ω_0^d_y(t), ω_0^d_z(t)]])

Вектор измерений:


In [ ]:
H_3, Js_3, Jn_3, Δ_3, report_3 = observe_system(n_c=1, 
                                                n_d=1, 
                                                is_c_move=True, 
                                                is_d_qw=True, 
                                                is_angles=True, 
                                                is_drag=True, 
                                                is_only_xz=False, 
                                                testprint=False,
                                                hand_written_deriv=None)

In [ ]:
H_3, Js_3, Jn_3, Δ_3, report_3 = observe_system(n_c=12, 
                                                n_d=1, 
                                                is_c_move=True, 
                                                is_d_qw=True, 
                                                is_angles=True, 
                                                is_drag=True, 
                                                is_only_xz=False, 
                                                testprint=False,
                                                hand_written_deriv=None)

In [ ]:
H_3, Js_3, Jn_3, Δ_3, report_3 = observe_system(n_c=1, 
                                                n_d=1, 
                                                is_c_move=True, 
                                                is_d_qw=True, 
                                                is_angles=True, 
                                                is_drag=True, 
                                                is_only_xz=True, 
                                                testprint=False,
                                                hand_written_deriv=None)

In [ ]:
H_5, Js_5, Jn_5, Δ_5, report_5 = observe_system(n_c=1, 
                                                n_d=1, 
                                                is_c_move=True, 
                                                is_d_qw=True, 
                                                is_angles=True, 
                                                is_drag=True, 
                                                is_only_xz=True, 
                                                testprint=False,
                                                hand_written_deriv=None)